# Patent Data

### Warning

This script has been modified to automate data download and environment setup. **Please ensure you're using your desired python environment before continuing**, and that you're running this from the GitHub repository root.

In [9]:
import pandas as pd
import os, sys, subprocess

# This does all the setup steps for the project - don't modify it
subprocess.run(['python3', '__init__.py'])

Installing required packages, creating necessary directories, and downloading the data...
PLEASE ENSURE YOU'RE USING YOUR DESIRED ENVIRONMENT!
Installing required packages...

--- DOWNLOADING RAW TABLES ---

[init]: Downloading g_patent.tsv...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  210M  100  210M    0     0  24.0M      0  0:00:08  0:00:08 --:--:-- 22.5M


[init]: Unzipping g_patent.tsv...
Archive:  /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_patent.tsv.zip
  inflating: /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_patent.tsv  

[init]: Downloading g_inventor_not_disambiguated.tsv...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  927M  100  927M    0     0  22.0M      0  0:00:41  0:00:41 --:--:-- 10.1M


[init]: Unzipping g_inventor_not_disambiguated.tsv...
Archive:  /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_inventor_not_disambiguated.tsv.zip
  inflating: /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_inventor_not_disambiguated.tsv  

[init]: Downloading g_location_not_disambiguated.tsv...


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  449M  100  449M    0     0  11.0M      0  0:00:40  0:00:40 --:--:-- 11.1M


[init]: Unzipping g_assignee_not_disambiguated.tsv...
Archive:  /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_assignee_not_disambiguated.tsv.zip
  inflating: /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_assignee_not_disambiguated.tsv  

[init]: Downloading g_wipo_technology.tsv...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.3M  100 45.3M    0     0  9855k      0  0:00:04  0:00:04 --:--:--  9.7M


[init]: Unzipping g_wipo_technology.tsv...
Archive:  /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_wipo_technology.tsv.zip
  inflating: /Users/aidan/Library/CloudStorage/SynologyDrive-aidanSync/School/Masters/Fall 2024/CSE6242/project/project-laptop/CSE6242-PatentProject/data/raw/g_wipo_technology.tsv  
[init]: All raw tables downloaded.
[init]: Initialization complete.


CompletedProcess(args=['python3', '__init__.py'], returncode=0)

In [ ]:
# Paths (modified to be relative to the current working directory)
patents_g_p   = os.path.join(os.getcwd(), 'data', 'raw', 'g_patent.tsv')
applicant_g_p = os.path.join(os.getcwd(), 'data', 'raw', 'g_inventor_not_disambiguated.tsv')
location_g_p  = os.path.join(os.getcwd(), 'data', 'raw', 'g_location_not_disambiguated.tsv')
assignee_g_p  = os.path.join(os.getcwd(), 'data', 'raw', 'g_assignee_not_disambiguated.tsv')
wipo_g_p      = os.path.join(os.getcwd(), 'data', 'raw', 'g_wipo_technology.tsv')

# Loading everything into memory takes over a minute
#patent information
patents_g = pd.read_csv(patents_g_p, sep='\t')

#applicant information (include multiple per patent doing seq = 0 should be primary)
applicant_g = pd.read_csv(applicant_g_p, sep='\t')

# raw location not disambiguated (e.g, no coordinates)
location_g = pd.read_csv(location_g_p, sep='\t')

#get organization
assignee_g = pd.read_csv(assignee_g_p, sep='\t')

#the sector and type of patent
wipo_g = pd.read_csv(wipo_g_p, sep='\t')

/var/folders/sx/6f330c0s1lx73cf7ntt5h3340000gp/T/ipykernel_85489/1994904581.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  patents_g = pd.read_csv(patents_g_p, sep='\t')


## Data Table Visualization 

In [11]:
## primary inventor...
prim_inventor = applicant_g[applicant_g['inventor_sequence'] == 0]

In [12]:
prim_inventor.head(5)

,patent_id,inventor_sequence,inventor_id,raw_inventor_name_first,raw_inventor_name_last,deceased_flag,rawlocation_id
0,D1006496,0,fl:we_ln:jiang-128,Wenjing,Jiang,0,30zgod902k0u495w9b2sb8xk9
2,6584128,0,fl:ri_ln:kroeger-1,Richard,Kroeger,FALSE,o2ema6bl3kkh6iwziprbzu4m9
3,4789863,0,fl:th_ln:bush-1,Thomas A.,Bush,FALSE,tkz70bmoqx88n1lfz3fl657lh
6,D474886,0,fl:th_ln:fleming-4,Thomas W.,Fleming,FALSE,cmf9hebugybql8ufffo47uwcz
8,7646155,0,fl:sa_ln:woods-2,Samuel G,Woods,FALSE,omi6wqlrblholsssk9qx0dz5b


In [13]:
applicant_g.head(5)

,patent_id,inventor_sequence,inventor_id,raw_inventor_name_first,raw_inventor_name_last,deceased_flag,rawlocation_id
0,D1006496,0,fl:we_ln:jiang-128,Wenjing,Jiang,0,30zgod902k0u495w9b2sb8xk9
1,12029253,4,fl:ei_ln:baumker-1,Eiko,Bäumker,0,468lxxjgdkuh0uonw27yj3t3f
2,6584128,0,fl:ri_ln:kroeger-1,Richard,Kroeger,FALSE,o2ema6bl3kkh6iwziprbzu4m9
3,4789863,0,fl:th_ln:bush-1,Thomas A.,Bush,FALSE,tkz70bmoqx88n1lfz3fl657lh
4,11161990,1,fl:ma_ln:boudreaux-4,Matthew F.,Boudreaux,False,up8ym622ssh09vpqv9rdsygs0


In [14]:
len(applicant_g['patent_id'].unique())

8979043

In [15]:
sum(applicant_g['inventor_sequence'] == 0)

8979043

### Patent

In [16]:
patents_g.head(5)

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml


### Location

In [17]:
location_g.head(5)

,rawlocation_id,location_id,raw_city,raw_state,raw_country
0,000005mtrirpdyrtlkfbffj0e,1d2251c8-16c8-11ed-9b5f-1234bde3cd05,Kanagawa,NaN,JP
1,00006fjnoq057no2s4bse374r,439af3dd-16c8-11ed-9b5f-1234bde3cd05,Dallas,TX,US
2,0000bztanu9rrtm943i8a7wry,8d09bc30-16c8-11ed-9b5f-1234bde3cd05,Osaka,NaN,JP
3,0000c07c-672a-48d9-a3bb-71c439ac17b3,280c9de4-16c8-11ed-9b5f-1234bde3cd05,NaN,NaN,DE
4,0000dpr8zljw34um291yal5ym,cf4944e5-16c7-11ed-9b5f-1234bde3cd05,Sugar Land,TX,US


### Wipo Technology

In [18]:
wipo_g.head(5)

,patent_id,wipo_field_sequence,wipo_field_id,wipo_sector_title,wipo_field_title
0,10000000,0,10,Instruments,Measurement
1,10000001,0,29,Mechanical engineering,Other special machines
2,10000001,1,12,Instruments,Control
3,10000002,0,21,Chemistry,"Surface technology, coating"
4,10000002,1,29,Mechanical engineering,Other special machines


## Merging Data

#### Assure all ids are string type is there is no issue when merging

In [21]:
patents_g.loc[:, 'patent_id']     = patents_g['patent_id'].astype('string')
prim_inventor.loc[:, 'patent_id'] = prim_inventor['patent_id'].astype('string')
wipo_g.loc[:, 'patent_id']        = wipo_g['patent_id'].astype('string')

#### Merge location and assignee

In [22]:
prim_assign  = assignee_g[assignee_g['assignee_sequence'] == 0]
organization = prim_assign[['patent_id', 'raw_assignee_organization']]

In [24]:
organization.loc[:, 'patent_id'] = organization['patent_id'].astype('string')

In [25]:
prim_inventor = pd.merge(prim_inventor, organization, on = 'patent_id', how = 'left')

In [26]:
prim_inventor.head(5)

,patent_id,inventor_sequence,inventor_id,raw_inventor_name_first,raw_inventor_name_last,deceased_flag,rawlocation_id,raw_assignee_organization
0,D1006496,0,fl:we_ln:jiang-128,Wenjing,Jiang,0,30zgod902k0u495w9b2sb8xk9,NaN
1,6584128,0,fl:ri_ln:kroeger-1,Richard,Kroeger,FALSE,o2ema6bl3kkh6iwziprbzu4m9,"Scientific-Atlanta, Inc."
2,4789863,0,fl:th_ln:bush-1,Thomas A.,Bush,FALSE,tkz70bmoqx88n1lfz3fl657lh,NaN
3,D474886,0,fl:th_ln:fleming-4,Thomas W.,Fleming,FALSE,cmf9hebugybql8ufffo47uwcz,"Bacou-Dalloz USA Safety, Inc."
4,7646155,0,fl:sa_ln:woods-2,Samuel G,Woods,FALSE,omi6wqlrblholsssk9qx0dz5b,Balck & Decker Inc.


In [27]:
# Ensure IDs are strings
location_g.loc[:, 'rawlocation_id']    = location_g['rawlocation_id'].astype('string')
prim_inventor.loc[:, 'rawlocation_id'] = prim_inventor['rawlocation_id'].astype('string')

# Filter location_g where raw_location_id exists in assignee_g.location_id to make merge faster since time complexity....idk if it makes that big of difference
#filtered_location_g = location_g[location_g['rawlocation_id'].isin(prim_inventor['rawlocation_id'])]
# ^ filtering is included in merge code

In [ ]:
# ~20s
location_assignee = pd.merge(prim_inventor, location_g, on='rawlocation_id', how='left')

In [29]:
location_assignee.head(4)

,patent_id,inventor_sequence,inventor_id,raw_inventor_name_first,raw_inventor_name_last,deceased_flag,rawlocation_id,raw_assignee_organization,location_id,raw_city,raw_state,raw_country
0,D1006496,0,fl:we_ln:jiang-128,Wenjing,Jiang,0,30zgod902k0u495w9b2sb8xk9,NaN,9d072d42-49af-11ed-9879-1234bde3cd05,Guizhou,NaN,CN
1,6584128,0,fl:ri_ln:kroeger-1,Richard,Kroeger,FALSE,o2ema6bl3kkh6iwziprbzu4m9,"Scientific-Atlanta, Inc.",NaN,NaN,NaN,NaN
2,4789863,0,fl:th_ln:bush-1,Thomas A.,Bush,FALSE,tkz70bmoqx88n1lfz3fl657lh,NaN,NaN,NaN,NaN,NaN
3,D474886,0,fl:th_ln:fleming-4,Thomas W.,Fleming,FALSE,cmf9hebugybql8ufffo47uwcz,"Bacou-Dalloz USA Safety, Inc.",1893009c-16c8-11ed-9b5f-1234bde3cd05,San Diego,CA,US


In [58]:
# Counting number of missing first and last names
no_f = sum(location_assignee['raw_inventor_name_first'].isna())
no_l = sum(location_assignee['raw_inventor_name_last'].isna())

print(f'Number of missing first names: {no_f}')
print(f'Number of missing last names: {no_l}')

Number of missing first names: 198
Number of missing last names: 1


In [30]:
missing_in_location_assignee = patents_g[~patents_g['patent_id'].isin(location_assignee['patent_id'])]
print("Rows in patents_g with no match in location_assignee:", len(missing_in_location_assignee))

missing_in_patents_g = location_assignee[~location_assignee['patent_id'].isin(patents_g['patent_id'])]
print("Rows in location_assignee with no match in patents_g:", len(missing_in_patents_g))


Rows in patents_g with no match in location_assignee: 1087
Rows in location_assignee with no match in patents_g: 0


In [32]:
# Checking for duplicate patent_ids (should be 0 because we're only taking primary inventors now)
print("Number of duplicate patent_ids in patents_g:", sum(patents_g.duplicated('patent_id')))

Number of duplicate patent_ids in patents_g: 0


In [34]:
patent_location_assignee = pd.merge(patents_g, location_assignee, on='patent_id', how='left')

### Getting Values with only Valid location

In [35]:
patent_location_assignee.head(5)

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename,inventor_sequence,inventor_id,raw_inventor_name_first,raw_inventor_name_last,deceased_flag,rawlocation_id,raw_assignee_organization,location_id,raw_city,raw_state,raw_country
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml,0.0,fl:jo_ln:marron-5,Joseph,Marron,FALSE,klkuujc1w3iik6xrp7c2wdvt7,Raytheon Company,198b0471-16c8-11ed-9b5f-1234bde3cd05,Manhattan Beach,CA,US
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml,0.0,fl:su_ln:lee-271,Sun-Woo,Lee,FALSE,95igh4mbo217kt9dmb2x8r3il,LS MTRON LTD.,6c4ba08f-16c8-11ed-9b5f-1234bde3cd05,Gunpo-si,NaN,KR
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml,0.0,fl:yu_ln:kim-79,Yun-Jo,Kim,FALSE,0jpghs5hm2z9y7c7t5wqcw9nk,"KOLON INDUSTRIES, INC.",755a6338-16c8-11ed-9b5f-1234bde3cd05,Yongin-si,NaN,KR
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml,0.0,fl:gu_ln:bergmann-3,Guido,Bergmann,FALSE,c9av9x4gxgkr3bvhc2dtrhsvz,KAUTEX TEXTRON GMBH & CO. KG,6c737da8-16c8-11ed-9b5f-1234bde3cd05,St. Augustin,NaN,DE
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml,0.0,fl:mi_ln:zubiriaelizondo-1,Miguel Jorge,Zubiria Elizondo,FALSE,v8ucm2trnfns3id8krhmw8bty,ZUBEX INDUSTRIAL SA DE CV,74ce52e1-16c8-11ed-9b5f-1234bde3cd05,Monterrey,NaN,MX


In [36]:
valid_locations = patent_location_assignee[patent_location_assignee['rawlocation_id'].notnull()]


In [39]:
print(f"So we have {len(valid_locations)} / {len(patent_location_assignee)} = {len(valid_locations) / len(patent_location_assignee) * 100:.2f}% valid locations.")
print(f"Dropping {len(patent_location_assignee) - len(valid_locations)} rows with missing locations...")
valid_locations = valid_locations.dropna(subset=['rawlocation_id'])

So we have 8978910 / 8980130 = 99.99% valid locations.
Dropping 1220 rows with missing locations.


In [40]:
len(valid_locations['patent_id'].unique())

8978910

### WIPO Type and Sector

This is where some work still needs to be done. The `utility` tags that come back from the below code are too general and apply to >99% of the patents in the file. We need to, per-patent, extract more detailed sector/industry categories from the wipo table. This is my next step.

In [41]:
prim_type = wipo_g[wipo_g['wipo_field_sequence'] == 0]

In [49]:
TechType_df = pd.merge(valid_locations, prim_type, on='patent_id', how='left')


In [50]:
len(TechType_df)

8978910

In [51]:
TechType_df.head(4)

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename,inventor_sequence,inventor_id,...,rawlocation_id,raw_assignee_organization,location_id,raw_city,raw_state,raw_country,wipo_field_sequence,wipo_field_id,wipo_sector_title,wipo_field_title
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml,0.0,fl:jo_ln:marron-5,...,klkuujc1w3iik6xrp7c2wdvt7,Raytheon Company,198b0471-16c8-11ed-9b5f-1234bde3cd05,Manhattan Beach,CA,US,0.0,10.0,Instruments,Measurement
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml,0.0,fl:su_ln:lee-271,...,95igh4mbo217kt9dmb2x8r3il,LS MTRON LTD.,6c4ba08f-16c8-11ed-9b5f-1234bde3cd05,Gunpo-si,NaN,KR,0.0,29.0,Mechanical engineering,Other special machines
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml,0.0,fl:yu_ln:kim-79,...,0jpghs5hm2z9y7c7t5wqcw9nk,"KOLON INDUSTRIES, INC.",755a6338-16c8-11ed-9b5f-1234bde3cd05,Yongin-si,NaN,KR,0.0,21.0,Chemistry,"Surface technology, coating"
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml,0.0,fl:gu_ln:bergmann-3,...,c9av9x4gxgkr3bvhc2dtrhsvz,KAUTEX TEXTRON GMBH & CO. KG,6c737da8-16c8-11ed-9b5f-1234bde3cd05,St. Augustin,NaN,DE,0.0,29.0,Mechanical engineering,Other special machines


In [52]:
sum(TechType_df['raw_country'].isna())

# Explore if there are any state or city information for the missing countries
missing_countries = TechType_df[TechType_df['raw_country'].isna()]
non_na = missing_countries[missing_countries['raw_state'].notna() | missing_countries['raw_city'].notna()]
non_na.head(5)
print(f"Number of rows with missing country but non-missing state or city: {len(non_na)}")
if len(missing_countries) != 0:
    print(f"This is {len(non_na) / len(missing_countries) * 100:.2f}% of the rows with missing countries, and {len(non_na) / len(TechType_df) * 100:.2f}% of the total rows.")
print(f"This is too few to be useful, so we're dropping these.")

TechType_df = TechType_df.dropna(subset=['raw_country'])

Number of rows with missing country but non-missing state or city: 1829
This is 91.82% of the rows with missing countries, and 0.02% of the total rows.
This is too few to be useful, so we're dropping these.


In [53]:
TechType_df.columns

Index(['patent_id', 'patent_type', 'patent_date', 'patent_title', 'wipo_kind',
       'num_claims', 'withdrawn', 'filename', 'inventor_sequence',
       'inventor_id', 'raw_inventor_name_first', 'raw_inventor_name_last',
       'deceased_flag', 'rawlocation_id', 'raw_assignee_organization',
       'location_id', 'raw_city', 'raw_state', 'raw_country',
       'wipo_field_sequence', 'wipo_field_id', 'wipo_sector_title',
       'wipo_field_title'],
      dtype='object')

In [54]:
### Dropping Unecessary Columns
df_patent_ID = TechType_df.drop(['patent_type', 'inventor_sequence', 'deceased_flag', 'wipo_kind','rawlocation_id','location_id','wipo_field_sequence'], axis = 1)

In [55]:
df_patent_ID.head(4)

,patent_id,patent_date,patent_title,num_claims,withdrawn,filename,inventor_id,raw_inventor_name_first,raw_inventor_name_last,raw_assignee_organization,raw_city,raw_state,raw_country,wipo_field_id,wipo_sector_title,wipo_field_title
0,10000000,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,20,0,ipg180619.xml,fl:jo_ln:marron-5,Joseph,Marron,Raytheon Company,Manhattan Beach,CA,US,10.0,Instruments,Measurement
1,10000001,2018-06-19,Injection molding machine and mold thickness c...,12,0,ipg180619.xml,fl:su_ln:lee-271,Sun-Woo,Lee,LS MTRON LTD.,Gunpo-si,NaN,KR,29.0,Mechanical engineering,Other special machines
2,10000002,2018-06-19,Method for manufacturing polymer film and co-e...,9,0,ipg180619.xml,fl:yu_ln:kim-79,Yun-Jo,Kim,"KOLON INDUSTRIES, INC.",Yongin-si,NaN,KR,21.0,Chemistry,"Surface technology, coating"
3,10000003,2018-06-19,Method for producing a container from a thermo...,18,0,ipg180619.xml,fl:gu_ln:bergmann-3,Guido,Bergmann,KAUTEX TEXTRON GMBH & CO. KG,St. Augustin,NaN,DE,29.0,Mechanical engineering,Other special machines


In [ ]:
dest_path = os.path.join(os.getcwd(), 'data', 'processed_patents.tsv')
df_patent_ID.to_csv(dest_path, sep='\t', index=False)